In [2]:
from utility_bfvariater import *
import utility_common as common

# creat a BF variater
er = BFVariater()

# 1. Central Value

In [3]:
def saveCentralValues(er):
    print( "save conteral values")

    contralValue = []
    for i in range(4):

        ndata = er.ndata[i]
        nmcbg = er.nmcbg[i].dot(np.ones(3))
        nfake = er.nfake[i]
        a = np.sum(er.a[i] * er.ca, axis=1)

        slv0 = BFSolver3D(a)
        br0 = slv0.solveQuadEqn(slv0.setMeasuredX(nData=ndata, nMcbg=nmcbg+nfake))
        contralValue.append(br0)
    contralValue = np.array(contralValue).reshape(12)

    baseDir = common.getBaseDirectory() 

    print(contralValue)

    np.save(baseDir+'/data/combine/contralValue',contralValue)
saveCentralValues(er)

save conteral values
[ 0.10952922  0.10757386  0.10391938  0.11200021  0.11023497  0.10355297
  0.10492754  0.10761074  0.10437442  0.10986088  0.10864141  0.10559611]


# 2. uncertainty

### 2.1 Statistical Uncertainty

In [4]:
def saveStatError(er):
    print( "relative statistical uncertainty [%]")

    stat_sigma, stat_covar = [],[]

    for source in ["data","mcbg","mcsg"]:
        sigma, variance_raw = er.errStat(source,returnCovar=True)
        # reshape 4x3x3 to 12x12
        variance = np.zeros([12,12])
        for c in range(4):
            for i in range(3):
                for j in range(3):
                    variance[c*3+i,c*3+j] = variance_raw[c,i,j]
                    
        stat_sigma.append(sigma)
        stat_covar.append(variance)

    stat_sigma = np.array(stat_sigma)  
    stat_covar = np.array(stat_covar) 

    er.io_printErrorForExcelFormat(stat_sigma.transpose(1,0,2))

    s = stat_sigma.shape
    stat_sigma = stat_sigma.reshape(s[0],s[1]*s[2])

    baseDir = common.getBaseDirectory() 
    np.save(baseDir+'/data/combine/stat_sigma',stat_sigma)
    np.save(baseDir+'/data/combine/stat_covar',stat_covar)

saveStatError(er)

relative statistical uncertainty [%]
0.553,0.516,1.218, 0.982,0.869,2.021, 0.710,0.550,1.444, 1.236,0.966,2.455
0.134,0.546,0.589, 0.137,0.515,0.636, 0.640,0.177,0.692, 0.595,0.166,0.635
0.266,0.226,0.571, 0.462,0.395,0.928, 0.323,0.264,0.690, 0.574,0.452,1.134


### 2.2 Systematical Uncertainty

In [5]:
def saveSystError(er):
    print( "relative systematics uncertainty [%]")
    syst_sigma, syst_covar = [],[]

    for source in ["BtmUp","BteUp"]:
        sigma = er.errConstent(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
    for source in ["lumin","mctt","mctw","mcw",'mcz','mcvv',
                   "fakee","fakemu","fakeetau","fakemutau"]:
        sigma = er.errSystem_crossSection(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
        
    for source in ["ERecoEff","EIDEff","MuEff","TauIDEff","JetToTauIDEff",
                   "EPt","MuPt","Tau0Pt","Tau1Pt","Tau10Pt"]:
        sigma = er.errSystem_downVariation(source)
        variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
        syst_sigma.append(sigma)
        syst_covar.append(variance)

    for source in ["Pileup","JES","JER","BTag","Mistag"]:
        sigma = er.errSystem_upDownVariation(source)
        variance = np.outer(sigma,sigma)
        syst_sigma.append(sigma)
        syst_covar.append(variance)
        
    
    syst_sigma = np.array(syst_sigma) 
    syst_covar = np.array(syst_covar) 

    er.io_printErrorForExcelFormat(syst_sigma.transpose(1,0,2))

    s = syst_sigma.shape
    syst_sigma = syst_sigma.reshape(s[0],s[1]*s[2])

    baseDir = common.getBaseDirectory() 
    np.save(baseDir+'/data/combine/syst_sigma',syst_sigma)
    np.save(baseDir+'/data/combine/syst_covar',syst_covar)

saveSystError(er)

relative systematics uncertainty [%]
0.005,0.049,0.074, 0.004,0.048,0.074, 0.007,0.048,0.073, 0.007,0.049,0.075
0.027,0.010,0.058, 0.028,0.010,0.058, 0.024,0.007,0.058, 0.024,0.008,0.060
0.151,0.279,0.035, 0.050,0.062,0.001, 0.148,0.143,0.066, 0.005,0.045,0.024
0.010,0.003,0.098, 0.003,0.010,0.055, 0.011,0.008,0.108, 0.009,0.006,0.003
0.011,0.003,0.103, 0.003,0.011,0.057, 0.012,0.009,0.113, 0.009,0.007,0.003
0.157,0.156,0.146, 0.051,0.051,0.039, 0.149,0.152,0.164, 0.048,0.047,0.072
0.293,1.410,0.142, 0.097,0.343,0.072, 0.872,0.268,0.054, 0.110,0.084,0.045
0.000,0.020,0.012, 0.001,0.009,0.002, 0.016,0.001,0.008, 0.006,0.001,0.001
0.000,0.000,0.000, 0.000,0.000,0.000, 0.374,0.376,0.767, 0.201,0.192,0.347
0.642,0.615,1.259, 0.418,0.400,0.697, 0.000,0.000,0.000, 0.000,0.000,0.000
0.000,0.000,0.000, 0.000,0.000,0.000, 0.257,0.345,1.864, 0.122,0.169,0.913
0.277,0.375,1.908, 0.164,0.219,1.134, 0.000,0.000,0.000, 0.000,0.000,0.000
0.397,0.036,0.030, 0.400,0.036,0.029, 0.024,0.023,0.011, 0.001,

# 3. output Latex Table

In [6]:
%run -i '/Users/zihengchen/Documents/Analysis/workplace/scripts/note_uncertainty.py'

In [ ]:
#     for source in ["e","mu","tauID",'jetMisTauID']:
#         sigma = er.errSystem_objectEff(source)
#         variance = np.outer(sigma.reshape(1,12),sigma.reshape(1,12))
#         syst_sigma.append(sigma)
#         syst_covar.append(variance)